In [ ]:
%cd ../src/init_geos_mom6
%pwd

## Download test data 
https://zenodo.org/records/15170073

In [ ]:
!wget -c https://zenodo.org/records/15170073/files/TEST_DATA_init_geos_mom6.zip
!unzip TEST_DATA_init_geos_mom6.zip
!ln -sf TEST_DATA_init_geos_mom6/INPUT/* .
!ls

## Generate T/S IC from the GEOS-S2S3 ODAS file
- Data is the ODAS analysis on uniform lat-lon grids

In [ ]:
!PYTHONPATH="../common" ./init_m6ts_from_m5odas.py ic_ts_m5odas.nc \
        --m5Ana ocean_temp_salt.res.nc \
        --m5Grid grid_spec.nc.mom5_LL1440x721

## Generate T/S IC from the CMEMS 
- Data can be found at (https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_PHY_ENS_001_031/description)
- 75-level, 0.25deg daily ocean analysis ensemble covering 80S to 90N
- 90S-80S are flooded through a relaxation

In [ ]:
# change "oras" to "glor" or "cglo" if not want to using ORAS5
!PYTHONPATH="../common" ./init_m6ts_from_cmems.py ic_ts_cmems.nc \
        --fnin ./cmems_mod_glo_phy-all_my_0.25deg_P1D-m-20230101.nc \
        --oname oras

## Generate T/S IC from the UFS replay
- Data can be found at (https://registry.opendata.aws/noaa-ufs-gefsv13replay-pds/)
- Model tripolar grid info is stored at the cycle 06z
- Use ocean history files (e.g., `ocn_2023_04_01_06.nc`) as the input.

In [ ]:
### Step 1: generate the regridder with a uniform resolution of 5 deg
!PYTHONPATH="../common" ./create_regridder.py wts.nc \
        --fngrd ufs.cpld.cpl.r.2016-01-02-10800.nc \
        --latname ocnExp_lat \
        --lonname ocnExp_lon \
        --res 5

In [ ]:
### Step 2: generate T/S IC 
# change to "--npes 75" when using 1 Milan node on Discover; 
# 
!PYTHONPATH="../common" ./init_m6ts_from_ufsreplay.py ic_ts_ufs.nc \
        --fnDiag ocn_2016_01_02_00.nc \
        --fnRgrdr wts.nc \
        --npes 8
